In [1]:
#Imports
import pandas as pd
import numpy as np
import plotly.express as px

## PEOPLE DATASET EXPLORATION

In [2]:
# Load the csv file
people_df = pd.read_csv('../../data/People.csv')

In [4]:
us_cities_info = pd.read_csv('../../data/us_cities_info.csv')

In [ ]:
# Print information
print('Info:')
people_df.info()

#Print columns
print('\nColumns:')
print(people_df.columns)

#Print shape
print('\nShape:')
print(people_df.shape)


In [ ]:
#Get unique values per non-numeric columns
for column in people_df.select_dtypes(exclude=[np.number]):
    print(f"Unique values in {column}:")
    print(people_df[column].unique())
    print("\n")

In [ ]:
#Check Duplicated Observations
people_df.duplicated().sum()

In [ ]:
# Check entries that 'CITY' column has a numeric value
people_df['CITY_numeric'] = pd.to_numeric(people_df['CITY'], errors='coerce')
numeric_count = people_df['CITY_numeric'].notna().sum()
print(numeric_count)

In [4]:
#Check if there are missing values and how many
print('Missing values per column:')
print(people_df.isna().sum())

# Print columns with missing values
columns_with_missing_values = people_df.columns[people_df.isnull().any()].tolist()
print('\nColumns with missing values:')
print(columns_with_missing_values)

Missing values per column:
PERSON_ID                     0
PERSON_TYPE                   0
RD_NO                         0
VEHICLE_ID                10696
CRASH_DATE                    0
CITY                     143920
STATE                    141976
SEX                        7154
AGE                      160981
SAFETY_EQUIPMENT           1410
AIRBAG_DEPLOYED           10452
EJECTION                   6705
INJURY_CLASSIFICATION       296
DRIVER_ACTION            109766
DRIVER_VISION            109884
PHYSICAL_CONDITION       109501
BAC_RESULT               108833
DAMAGE_CATEGORY               0
DAMAGE                    74309
dtype: int64

Columns with missing values:
['VEHICLE_ID', 'CITY', 'STATE', 'SEX', 'AGE', 'SAFETY_EQUIPMENT', 'AIRBAG_DEPLOYED', 'EJECTION', 'INJURY_CLASSIFICATION', 'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION', 'BAC_RESULT', 'DAMAGE']


In [3]:
people_df['CITY'].value_counts().to_csv('cities.csv')

In [ ]:
#Get value counts for columns with missing values
for col in people_df[columns_with_missing_values]:
    print(people_df[col].value_counts())
    print('------')

In [7]:
# Retrieve columns that provide a value containing "Unknown" substring to indicate when the information for that column is not known.
for col in people_df[columns_with_missing_values]:
    if people_df[col].astype(str).str.contains('unknown', case=False, na=False).any():
        print(f'Column name: {col}')
        print('---')
        people_df[col].value_counts().to_csv(f'{col} value counts')
        

Column name: CITY
---
Column name: SAFETY_EQUIPMENT
---
Column name: AIRBAG_DEPLOYED
---
Column name: EJECTION
---
Column name: DRIVER_ACTION
---
Column name: DRIVER_VISION
---
Column name: PHYSICAL_CONDITION
---
Column name: BAC_RESULT
---


'PEOPLE' columns that provide an 'UNKNOWN' value
From the columns with missing values:

1. CITY
2. SAFETY_EQUIPMENT
3. AIRBAG_DEPLOYED
4. EJECTION
5. DRIVER_ACTION
6. DRIVER_VISION
7. PHYSICAL CONDITION
8. BAC_RESULT

## Queries to explore potential value pairings

In [34]:
# Check if there are observations where city is known but state is NaN
print('Count of observations that have the \"CITY\" column filled but not the \"STATE\" column:')
count_rows_without_states= people_df[columns_with_missing_values].query('CITY == CITY and STATE != STATE')[['CITY', 'STATE']].shape[0]
print(count_rows_without_states)
if count_rows_without_states > 0:
    print('First 5 rows that have the \"CITY\" column filled but not the \"STATE\" column:')
    print(people_df[columns_with_missing_values].query('CITY == CITY and STATE != STATE')[['CITY', 'STATE']].head(5))

Count of observations that have the "CITY" column filled but not the "STATE" column:
9005
First 5 rows that have the "CITY" column filled but not the "STATE" column:
                 CITY STATE
727           CHICAGO   NaN
755           CHICAGO   NaN
757   LAURIER-STATION   NaN
996           CHICAGO   NaN
1003          CHICAGO   NaN


In [33]:
# Check if there are observations where state is known but city is NaN
print('Count of observations that have the \"STATE\" column filled but not the \"CITY\" column:')
count_rows_without_cities = people_df[columns_with_missing_values].query('CITY != CITY and STATE == STATE')[['CITY', 'STATE']].shape[0]
print(count_rows_without_cities)
if count_rows_without_cities > 0:
    print('First 5 rows that have the \"STATE\" column filled but not the \"CITY\" column:')
    print(people_df[columns_with_missing_values].query('CITY != CITY and STATE == STATE')[['CITY', 'STATE']].head(5))

Count of observations that have the "STATE" column filled but not the "CITY" column:
10949
First 5 rows that have the "STATE" column filled but not the "CITY" column:
    CITY STATE
39   NaN    IL
61   NaN    IL
132  NaN    XX
199  NaN    IL
200  NaN    IL


In [ ]:
# Check if there are observations where the type is "PASSENGER" and ""DRIVER_ACTION" or "DRIVER_VERSION" are filled
print('Count of observations that have the \"PERSON_TYPE\" as "PASSENGER" and the columns  \"DRIVER_ACTION\" or \"DRIVER_VERSION\" filled:')
count_passenger_with_driver_info = people_df.query('PERSON_TYPE == "PASSENGER" and (DRIVER_ACTION == DRIVER_ACTION or DRIVER_VISION == DRIVER_VISION)').shape[0]
print(count_passenger_with_driver_info)
if count_passenger_with_driver_info > 0:
    print(people_df.query('PERSON_TYPE == "PASSENGER" and (DRIVER_ACTION == DRIVER_ACTION or DRIVER_VISION == DRIVER_VISION)').head(5))

In [ ]:
# Check if there are observations where the "AGE" < 10  and "PERSON_TYPE" is "DRIVER" (Anomalies)
print('Count of observations that have the "AGE" < 10 and \"PERSON_TYPE\" as "DRIVER":')
count_age_0 = people_df.query('AGE  < 10 and PERSON_TYPE == "DRIVER"').shape[0]
print(count_age_0)
if count_age_0 > 0:
    print(people_df.query('AGE < 10 and PERSON_TYPE == "DRIVER"')[['AGE', 'PERSON_TYPE']])

## Plots

In [ ]:
#List of columns to plot
columns_to_plot = [
'PERSON_TYPE',
'STATE',
'SEX',
'SAFETY_EQUIPMENT',
'AIRBAG_DEPLOYED',
'EJECTION',
'DRIVER_ACTION',
'DRIVER_VISION',
'PHYSICAL_CONDITION',
'BAC_RESULT',
'DAMAGE_CATEGORY'
]


#Plot each column
for column in columns_to_plot:
    fig = px.bar(people_df, 
                 x=column, 
                 labels={'x': column, 'y': 'Frequency'},
                 title=f'Distribution of {column}')
    # Customize hover information
    fig.update_traces(hovertemplate='Frequency: %{y}<extra></extra>')
    fig.update_xaxes(tickangle=45)
    fig.show()

In [ ]:
fig = px.histogram(people_df, x='AGE', 
                   title='Age Distribution',
                   labels={'AGE': 'Age', 'count': 'Frequency'},
                   nbins=20) 
# Customize hover information
fig.update_traces(hovertemplate='Frequency: %{y}<extra></extra>')
fig.show()

## Potential changes

After the data exploration on "People" dataframe, I believe we should:
1. Use "CITY" column to determine "STATE" column when the latter is empty. ✅
2. Make following mapping for 'SEX' values: U & NaN values -> "U"(unknown)✅
3. For all the columns that already provide an "unknown" possible value, fill NaN observations with this value.✅
4. Convert "VEHICLE_ID"  & "AGE" to an integer. ✅
5. Add new value 'N/A' ('NON APPLICABLE') for observations that the person_type is passenger and we have missing values on columns that regard the driver ('DRIVER_VISION' & 'DRIVER_ACTION').✅
6. Split the 'CRASH_DATE' into 'DAY', 'MONTH', 'YEAR', 'TIME' columns✅
7. Introduce either a sentinel value or 'NaN' value for 'AGE' for observations when "AGE" < 10  and "PERSON_TYPE" is "DRIVER"✅
8. Make 'STATE' column have the value 'Unknown' when 'CITY' is 'UNKNOWN' or STATE == 'XX'.✅
9. Set 'CITY' column have the value 'Unknown' when 'city' has numeric value, length < 2 or starts with UNK✅